In [3]:
# base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Any, Dict, List

# Загрузка данных.

In [4]:
import gdown

url = "https://drive.google.com/file/d/1a9eo-9XzqZ1XFBaqfJHdecoB-oG-ch5B/view?usp=sharing"
output = "data.zip"
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1a9eo-9XzqZ1XFBaqfJHdecoB-oG-ch5B
From (redirected): https://drive.google.com/uc?id=1a9eo-9XzqZ1XFBaqfJHdecoB-oG-ch5B&confirm=t&uuid=54842b3f-99b0-409b-b7e6-b1b09c345877
To: /content/data.zip
100%|██████████| 26.6M/26.6M [00:00<00:00, 34.8MB/s]


'data.zip'

In [5]:
!mkdir data
!mkdir data/raw

In [6]:
!unzip data.zip -d data/raw

Archive:  data.zip
  inflating: data/raw/data_final_for_dls_new.jsonl  
  inflating: data/raw/data_final_for_dls_eval_new.jsonl  


In [7]:
!rm data.zip

In [8]:
data_eval = pd.read_json("data/raw/data_final_for_dls_eval_new.jsonl", lines=True)
data = pd.read_json("data/raw/data_final_for_dls_new.jsonl", lines=True)

data.drop(columns='relevance', inplace=True)
data_eval.drop(columns='relevance', inplace=True)

data.rename(columns={'relevance_new': 'relevance'}, inplace=True)
data_eval.rename(columns={'relevance_new': 'relevance'}, inplace=True)

# ВАЖНО! Так как у нас eval - просто первые 570 строк из всех данных.
data_clean = data.iloc[570:]

# Построение агента.

## 1. Define tools and model

In [9]:
!pip install -U langgraph
!pip install -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.8
    Uninstalling langchain-core-1.2.8:
      Successfully uninstalled langchain-core-1.2.8
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.8
    Uninstalling langchain-1.2.8:
      Successfully uninstalled langchain-1.2.8


In [10]:
!pip install 'ollama>=0.6.0'

### Каркас агента

In [11]:
DEBUG = 1

In [20]:
from __future__ import annotations

import json
from typing import TypedDict, Literal, Optional, Dict, Any, List, Sequence, Tuple


from langgraph.graph import StateGraph, START, END

Relevance = Literal[0.0, 0.1, 1.0]

class Organization(TypedDict, total=False):
    name: str
    category: str
    address: str
    description: str
    reviews: str

class AgentState(TypedDict, total=False):
    # input
    id: str
    query: str
    organization: Organization

    # control
    web_tries: int
    max_web_tries: int

    # router output
    can_decide_now: bool
    router_reason: str

    # web pipeline
    web_search_query: str
    web_search_result: str
    org_web_evidence: str  # накопленный “факт-лист” из веба

    # final output
    relevance: Relevance
    reason: str


### Удобный парсинг в json

In [13]:
def _extract_json_object(text: str) -> Dict[str, Any]:
    """
    Минимальная и практичная вырезка JSON-объекта из ответа LLM.
    Ожидаем, что модель вернёт JSON-объект.
    """
    text = text.strip()
    # Попытка прямого парсинга
    try:
        return json.loads(text)
    except Exception:
        pass

    # Попытка вырезать по первым/последним фигурным скобкам
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        chunk = text[start : end + 1]
        return json.loads(chunk)

    raise ValueError("Не удалось распарсить JSON из ответа модели")


def _clamp_relevance(x: Any) -> Relevance:
    # Допускаем, что LLM вернёт число или строку.
    try:
        v = float(x)
    except Exception:
        return 0.0
    if v == 1.0:
        return 1.0
    if v == 0.1:
        return 0.1
    return 0.0

### Prompts

In [22]:
ROUTER_PROMPT = """\
Ты являешься роутером в системе оценки релевантности организаций на картах по запросу пользователя.

Вход:
- query: поисковый запрос пользователя
- organization: данные организации (name, category, address, description, reviews)
- web_evidence: дополнительная информация из интернета (если есть)

Задача: определить, можно ли по текущей информации однозначно принять решение о релевантности
(0.0 / 0.1 / 1.0) без дополнительного веб-поиска.
Описание меток:
1.0 — Релевантно. Организация напрямую соответствует намерению запроса пользователя.
0.1 — Организация может удовлетворить запрос косвенно или как запасной вариант.
0.0 — Нерелевантно. Организация не соответствует намерению запроса.

Критерии "можно решить сейчас":
- Достаточно фактов, чтобы уверенно поставить 1.0 или 0.0
- Нет критически недостающих атрибутов, без которых решение будет гаданием
- Если category/description/reviews явно описывают другой тип места и противоречат intent запроса - can_decide_now=true (обычно 0.0).
Критерии "нельзя решить сейчас"
- Если запрос явно требует внешнего подтверждения (например "мишлен звезда", "сертификация", "официальный дилер", "лицензия", "2025"),
  и в данных этого нет — обычно НЕЛЬЗЯ решить сейчас.
- Если запрос содержит конкретный продукт/услугу (кофе, пиво, доставка, 24 часа, парковка) и в description/reviews нет прямого подтверждения — нельзя решить сейчас.

Запомни:
- Веб-поиск нужен только если есть признаки, что нужная услуга может быть, но не подтверждена.
- Если отсутствие подтверждения сочетается с сильной несовместимостью по категории — это достаточно для 0.0, веб не нужен

Верни СТРОГО валидный JSON-объект:
{
  "can_decide_now": true|false,
  "reason": "кратко, до 12 слов"
}
"""

MAKE_SEARCH_QUERY_PROMPT = """\
Ты формируешь запрос для веб-поиска, чтобы проверить недостающий факт для оценки релевантности организации на картах широкому запросу пользователя.

Дано:
- user query
- organization (name, category, address)
- router_reason: почему не хватает информации

Сформируй один поисковый запрос (одной строкой), максимально конкретный.
Используй имя + адрес/город, добавь ключевые слова проверяемого факта.

Учти, что далее будет производиться поиск в интернете -> чанкование документов -> поиск топ 5 самых близких чанков по косинусной схожести твоему запросу.
Поэтому
- составь запрос максимально корректно
- старайся указать ключевые слова в самом начале

Верни СТРОГО JSON:
{
  "web_search_query": "..."
}
"""

few_shot_basic = """Например
Дана организация - ресторан Erwin RekaMoreOkean Москва Кутузовский проспект. Запрос пользователя: мишленовские рестораны в москве. router_reason: нет инофрмации о звезде мишлена
Тогда сформируй примерно такой запрос:  "web_search_query": "Звезда мишлен Erwin RekaMoreOkean Москва, Кутузовский проспект"
"""

CLASSIFIER_PROMPT_SINGLE = """\
Ты являешься системой оценки релевантности организаций на картах для поисковой выдачи по запросу пользователя.

Твоя задача — определить релевантность организации относительно поискового запроса пользователя.

---

Входные данные (один кейс):

{
  "id": "<уникальный идентификатор кейса>",
  "query": "<поисковый запрос пользователя>",
  "organization": {
    "name": "<название>",
    "category": "<категория / рубрика>",
    "address": "<адрес>",
    "description": "<описание>",
    "reviews": "<отзывы>",
    "web_evidence": "<доп. факты из веба, если есть>"
  }
}

---

Классы релевантности (строго):
1.0 — Полностью релевантно
Организация напрямую соответствует намерению запроса пользователя.

0.1 — Частично релевантно (редко)
Организация может удовлетворить запрос косвенно или как запасной вариант.

0.0 — Нерелевантно
Организация не соответствует намерению запроса.

Правила:
- Используй только данные текущего кейса
- Не додумывай факты
- 0.1 ставь редко, только если явно "косвенно"

Приоритет факторов (по убыванию важности):
1. Категория организации
2. Название организации
3. description и reviews
4. Адрес (учитывается только при наличии географического запроса)
5. web_evidence - если есть, значит модель уже обрабатывала запрос, не нашла нужных данных для верного ответа, обратилась в веб

Если есть web_evidence:
- Учти, что этот текст получен таким методом: веб поиск -> при помощи эмбеддингов получены топ-k чанков к уточняющему запросу

Формат ответа: верни СТРОГО валидный JSON-объект:
{
  "id": "<id из входа>",
  "relevance": 1.0 | 0.1 | 0.0,
  "reason": "<не более 12 слов, одно предложение, без общих формулировок>"
}
"""

### Nodes

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def router_node(state: AgentState, llm) -> dict:
    """
    Router node
    """
    org = state["organization"]
    payload = {
        "query": state["query"],
        "organization": org,
        "web_evidence": state.get("org_web_evidence", "")
    }

    msg = ROUTER_PROMPT + "\n\nINPUT:\n" + json.dumps(payload, ensure_ascii=False)
    if DEBUG:
        print(f"[LOG] Node: Router.  Prompting: {payload}")
    raw = llm.invoke(msg).content  # ожидаем объект с can_decide_now/reason

    obj = _extract_json_object(raw)

    can = bool(obj.get("can_decide_now", False))
    if DEBUG:
        print(f"\t[LOG] Model verdict: can decide now - {can}")
        print(f"\t[LOG] Model verdict: {str(obj.get("reason", ""))}")

    # если лимит веба исчерпан — принудительно решаем
    if state.get("web_tries", 0) >= state.get("max_web_tries", 2):
        can = True
        if DEBUG:
            print(f"\t[LOG] Web Limit. Forcing model to classify.")

    return {
        "can_decide_now": can,
        "router_reason": str(obj.get("reason", "")),
    }



def make_search_query_node(state: AgentState, llm) -> dict:
    """
    Generate a search query to get information

    """

    if DEBUG:
        print(f"[LOG] Node: Make search query.")
    org = state["organization"]
    payload = {
        "query": state["query"],
        "organization": {
            "name": org.get("name", ""),
            "category": org.get("category", ""),
            "address": org.get("address", ""),
        },
        "router_reason": state.get("router_reason", "")
    }

    msg = MAKE_SEARCH_QUERY_PROMPT + "\n\nINPUT:\n" + json.dumps(payload, ensure_ascii=False)
    raw = llm.invoke(msg).content
    obj = _extract_json_object(raw)

    if DEBUG:
        print(f"\t[LOG] Model generated Query: {obj.get('web_search_query', '')}")

    q = str(obj.get("web_search_query", "")).strip()
    if not q:
        # fallback: простая эвристика
        q = f'{org.get("name","")} {org.get("address","")} {state["query"]}'.strip()

    return {"web_search_query": q}


# --------------------------------------
# --- методы для удобного веб поиска ---
# --------------------------------------
# (тут специально выделил, не думайте что гпт)

class WebSearchResult(TypedDict, total=False):
    content: str
    url: str
    title: str

def ollama_web_search(query: str, max_results=5) -> List[WebSearchResult]:
    """
    Выполнение поиска с использованием ollama api
    Здесь надо учесть, что лимит может быть исчерпан!

    """
    response = ollama.web_search(query, max_results)['results']

    web_search_result: List[WebSearchResult] = []

    for dict_res in response:
        web_search_result.append({
            "content": dict_res['content'],
            "url": dict_res['url'],
            "title": dict_res['title']
        })

    return web_search_result

class RetrievedChunk(TypedDict):
    text: str
    score: float
    url: str
    title: str

def retrieve_top_chunks_on_the_fly(
    query: str,
    web_results: Sequence[Dict[str, Any]],
    embedding_model,
    top_k: int = 5,
    min_chunk_len: int = 80,
    chunk_size: int = 380,
    chunk_overlap: int = 60,
    batch_size: int = 64,
    max_total_chunks: int = 2000,
    lexical_prefilter_topn: int = 800) -> List[RetrievedChunk]:

    query = (query or "").strip()
    if not query or not web_results:
        return []

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
    )

    chunks: List[str] = []
    metas: List[Tuple[str, str]] = []

    # 1) чанкование
    for d in web_results:
        content = str(d.get("content", "") or "").strip()
        if not content:
            continue

        url = str(d.get("url", "") or "")
        title = str(d.get("title", "") or "")

        parts = splitter.split_text(content)
        for p in parts:
            p = p.strip()
            if len(p) < min_chunk_len:
                continue
            chunks.append(p)
            metas.append((url, title))
            if len(chunks) >= max_total_chunks:
                break
        if len(chunks) >= max_total_chunks:
            break

    if not chunks:
        return []

    # 2) дешёвый предфильтр по лексике (на данный момент убран из-за ненадобности)

    # 3) embeddings (E5: префиксы query:/passage:)

    q_text = "query: " + query
    p_texts = ["passage: " + c for c in chunks]

    qv = embedding_model.encode(
        [q_text],
        batch_size=1,
        show_progress_bar=False,
        convert_to_numpy=True,
        normalize_embeddings=True,
    ).astype(np.float32)  # (1, dim)

    X = embedding_model.encode(
        p_texts,
        batch_size=batch_size,
        show_progress_bar=False,
        convert_to_numpy=True,
        normalize_embeddings=True,
    ).astype(np.float32)  # (n, dim)

    # 4) cosine similarity = dot
    sims = (X @ qv.T).reshape(-1)

    k = min(top_k, len(chunks))
    idx = np.argpartition(-sims, kth=k - 1)[:k]
    idx = idx[np.argsort(-sims[idx])]

    out: List[RetrievedChunk] = []
    for j in idx:
        url, title = metas[int(j)]
        out.append({
            "text": chunks[int(j)],
            "score": float(sims[int(j)]),
            "url": url,
            "title": title,
        })

    return out



# конец методов для веб поиска
# ----------------------------


def web_search_node(state: AgentState) -> dict:
    """
    Нода выполняет запрос к LLM для формирования правильного запроса.
    В конце увеличивает счетчик попыток поиска.
    """
    q = state["web_search_query"]

    if DEBUG:
        print(f"[LOG] Node: web_search_node.")
        print(f"\t[LOG] Starting searching")

    result = ollama_web_search(q)

    if DEBUG:
        print(f"\t[LOG] Ending searching.")

    tries = int(state.get("web_tries", 0)) + 1
    return {
        "web_search_result": result,
        "web_tries": tries,
    }


def augment_context_node(state: AgentState, model_embeddings) -> dict:
    """
    Добавляем веб-результат в накопленное поле org_web_evidence.
    Внимание! Тут старый результат поиска если он есть удаляется.
    Причина: старый результат нам не нужен, видимо запрос модели был неккоректен.

    """
    if DEBUG:
        print(f"[LOG] Node: augment_context_node")
        print(f"\t[LOG] Starting finding chunks...")


    web_results = state.get("web_search_result") or []

    top_chunks = retrieve_top_chunks_on_the_fly(
        query=state.get("web_search_query", ""),
        web_results=web_results,
        embedding_model=model_embeddings,
        top_k=5,
        min_chunk_len = 80,
        chunk_size=356,
        chunk_overlap=60,
        batch_size=64,
        max_total_chunks=2000
    )

    if DEBUG:
        print(f"\t[LOG] Chunks finded: {top_chunks}")

    return {"org_web_evidence": top_chunks}


def classify_node(state: AgentState, llm) -> dict:

    if DEBUG:
        print(f"[LOG] Node: classify_node")

    org = dict(state["organization"])
    org["web_evidence"] = state.get("org_web_evidence", "")

    current_case = {
        "id": state["id"],
        "query": state["query"],
        "organization": org
    }

    msg = CLASSIFIER_PROMPT_SINGLE + "\n\nINPUT:\n" + json.dumps(current_case, ensure_ascii=False)
    raw = llm.invoke(msg).content

    obj = _extract_json_object(raw)

    rel = _clamp_relevance(obj.get("relevance"))
    reason = str(obj.get("reason", "")).strip()


    if DEBUG:
        print(f"\t[LOG] Node: classify_node. Model verdict: relevance {rel}, reason {reason}")

    return {
        "relevance": rel,
        "reason": reason,
    }

### Routing function

In [48]:
def route_after_router(state: AgentState) -> Literal["classify", "make_search_query"]:
    return "classify" if state.get("can_decide_now", False) else "make_search_query"

### Build graph

In [49]:
from sentence_transformers import SentenceTransformer

def build_graph(llm):
    g = StateGraph(AgentState)
    embedding_model = SentenceTransformer("intfloat/multilingual-e5-small")

    # nodes (оборачиваем, чтобы прокинуть llm)
    g.add_node("router", lambda s: router_node(s, llm))
    g.add_node("make_search_query", lambda s: make_search_query_node(s, llm))
    g.add_node("web_search", web_search_node)
    g.add_node("augment_context", lambda s: augment_context_node(s, embedding_model))
    g.add_node("classify", lambda s: classify_node(s, llm))

    # edges
    g.add_edge(START, "router")
    g.add_conditional_edges("router", route_after_router, {
        "classify": "classify",
        "make_search_query": "make_search_query",
    })

    g.add_edge("make_search_query", "web_search")
    g.add_edge("web_search", "augment_context")
    g.add_edge("augment_context", "router")  # цикл

    g.add_edge("classify", END)

    return g.compile()

### Запуск

In [50]:
import os
from google.colab import userdata

os.environ["OLLAMA_API_KEY"] = userdata.get("OLLAMA_API_KEY")
os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")

In [27]:
!pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.3 MB/s eta 0:00:00


In [51]:
from langchain_openai import ChatOpenAI

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
model_name = 'openrouter/pony-alpha'

llm = ChatOpenAI(
    model=model_name, # Specify a model available on OpenRouter
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
    temperature=0
)

graph = build_graph(llm)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


### Подготовка данных

In [52]:
def process_columns(df, names_num=3, prices_num=10):
    """обработка каждого столбца"""

    # Обработка имен
    df['name'] = df['name'].apply(
        lambda x: " ; ".join(str(x).split(';')[:names_num]) if pd.notna(x) else None
    )

    # Обработка цен
    df['prices_summarized'] = df['prices_summarized'].apply(
        lambda x: " | ".join(str(x).split('|')[:prices_num]) if pd.notna(x) else None
    )

    # Обработка отзывов (берем первый элемент до первого \n)
    df['reviews_summarized'] = df['reviews_summarized'].apply(
        lambda x: str(x).split('|')[0].split('\n')[0] if pd.notna(x) else None
    )

    return df

data_clean = process_columns(data_clean)
data_eval = process_columns(data_eval)


/tmp/ipython-input-2205466849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].apply(
/tmp/ipython-input-2205466849.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prices_summarized'] = df['prices_summarized'].apply(
/tmp/ipython-input-2205466849.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [58]:
data_clean.sample(5)

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,reviews_summarized,relevance
24388,ресторан с кабинками в москве,"Москва, Малый Черкасский переулок, 2",Mandarin Combustible ; Mandarin ; ...,Ресторан,25747942664,Ресторан предлагает разнообразные блюда азиатс...,Организация занимается предоставлением услуг в...,0.0
1615,восстановление пластика в машине омск,"Омск, Енисейская улица, 1М",Сапфир ; Жестяно-малярный цех,Кузовной ремонт,89569568291,None,"Организация занимается кузовным ремонтом, вклю...",0.0
26425,пивной ресторан сочи набережная,"Краснодарский край, Сочи, Черноморская улица, 3к7",Harat's ; Harat’s pub ; Harat's Pub,"Бар , паб",1400796318,None,"Организация Harat's — это бар, паб в Сочи, изв...",0.1
11679,шиномонтаж рядом,"Тамбов, Октябрьский район",Шиномонтаж,Шиномонтаж,108844291186,None,None,1.0
32001,кафе ролл,"Москва, Преображенская улица, 5/7",Тануки ; Tanuki ; Лидер,Ресторан,1237760768,Ресторан «Тануки» предлагает широкий выбор суш...,Организация занимается предоставлением услуг в...,1.0


In [63]:
# Get Example
def get_template_from_row(row) -> AgentState:

    organizatrion: Organization = {
            "name": row['name'],
            "category": row['normalized_main_rubric_name_ru'],
            "address": row['address'],
            "description": row['prices_summarized'] if row['prices_summarized'] is not None else 'None',
            "reviews": row['reviews_summarized'] if row['reviews_summarized'] is not None else 'None'
    }

    agent_template: AgentState = {
        "id" : row.name,
        "query" : row['Text'],
        "organization": organizatrion,

        "web_tries": 0,
        "max_web_tries": 1,
    }
    real_relevance = row['relevance']

    return agent_template, real_relevance

# 21994 - dogs +
# 26238 - bk kofee - модель идет в веб чтобы удостовериться что в бургер кинге есть коффе + (ошибка там надо 1 а он выводит 0.1 и в целом я с ним согласен) -
# 717 - ohr web - модель идет в веб чтобы глянуть есть ли одежда для охраны + (уже не идет в веб)
# 28801 - модель идет в веб, хотя не надо бы там нет зала борьбы + исправлено
# 20353 - модель идет в веб, хотя понятно что не надо - модель в конце еще и ошибается -
# 24388 - надо 0
# 26425 - надо 0.1
IND = 20353
init_state, real_relevance = get_template_from_row(data_clean.loc[IND])
print(init_state)
print(real_relevance)


{'id': 20353, 'query': 'кабельное телевидение и интернет во владимире', 'organization': {'name': 'Райon    ;     РайОN, Офис    ;     РайОN', 'category': 'Интернет-провайдер', 'address': 'Владимир, Студенческая улица, 5А', 'description': 'None', 'reviews': 'Организация «РайOn» предоставляет услуги интернет-провайдера. Отзывы смешанные: один положительный, один нейтральный и один отрицательный. Хвалят грамотных сотрудников. Критикуют отсутствие ответов на звонки в рабочее время. '}, 'web_tries': 0, 'max_web_tries': 1}
0.0


In [60]:
data[data['Text'] == init_state.get("query", "")]

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,reviews_summarized,relevance
6304,ресторан с кабинками в москве,"Москва, 1-я Тверская-Ямская улица, 2с1",Соло; Karaoke Solo; Solo; Караоке Соло на Маяк...,Караоке-клуб,15806185341,"Караоке-клуб, ночной клуб и бар, предлагающий ...",Организация занимается проведением караоке-веч...,1.0
10916,ресторан с кабинками в москве,"Москва, 1-й Красногвардейский проезд, 21, стр. 2",City Voice; صوت المدينة; Гарадскі голас; קול ה...,Караоке-клуб,195270796229,"City Voice предлагает широкий выбор роллов, су...","City Voice — караоке-клуб и ночной клуб, получ...",0.0
15278,ресторан с кабинками в москве,"Москва, Цветной бульвар, 34",Fillary,Караоке-клуб,55331208829,Ресторан и караоке-клуб Fillary предлагает раз...,Организация занимается проведением караоке-веч...,0.0
24388,ресторан с кабинками в москве,"Москва, Малый Черкасский переулок, 2",Mandarin Combustible; Mandarin; Мандарин бар; ...,Ресторан,25747942664,Ресторан предлагает разнообразные блюда азиатс...,Организация занимается предоставлением услуг в...,0.0
24985,ресторан с кабинками в москве,"Москва, проспект 60-летия Октября, 20",Апрель; April; Кафе-караоке Апрель; Aprel,Ресторан,70615511830,None,Организация «Апрель» занимается ресторанным би...,1.0
25214,ресторан с кабинками в москве,"Москва, улица Зацепский Вал, 4с2",Соло; Solo,Караоке-клуб,212932149242,"Караоке-клуб, ночной клуб и бар предлагают кар...","Организация «Соло» — это караоке-клуб, ночной ...",0.1
25991,ресторан с кабинками в москве,"Москва, Малый Путинковский переулок, 1/2с1",Эхо; Ekho; Eho; Караоке-бар Эхо; Плаза; Эхо на...,Караоке-клуб,1168535329,Караоке-клуб «Эхо» предлагает широкий выбор ал...,"Организация «Эхо» — это караоке-клуб с кафе, г...",0.1


### Запуск агента

In [64]:
out = graph.invoke(init_state)
print(out["relevance"], out["reason"])

[LOG] Node: Router.  Prompting: {'query': 'кабельное телевидение и интернет во владимире', 'organization': {'name': 'Райon    ;     РайОN, Офис    ;     РайОN', 'category': 'Интернет-провайдер', 'address': 'Владимир, Студенческая улица, 5А', 'description': 'None', 'reviews': 'Организация «РайOn» предоставляет услуги интернет-провайдера. Отзывы смешанные: один положительный, один нейтральный и один отрицательный. Хвалят грамотных сотрудников. Критикуют отсутствие ответов на звонки в рабочее время. '}, 'web_evidence': ''}
	[LOG] Model verdict: can decide now - False
	[LOG] Model verdict: Нет подтверждения наличия услуги кабельного ТВ
[LOG] Node: Make search query.
	[LOG] Model generated Query: РайОN Владимир кабельное телевидение
[LOG] Node: web_search_node.
	[LOG] Starting searching
	[LOG] Ending searching.
[LOG] Node: augment_context_node
	[LOG] Starting finding chunks...
	[LOG] Chunks finded: [{'text': 'РайON - услугидоступавинтернетицифровоготелевидениявг. Владимире* [&#128222;+7 (49

### Промежуточные выводы:

1. Агент хорошо справляется с явными и простыми запросами.
2. Путем промпт инжиниринга агент теперь старается редко ходить в веб.
3. Когда нужно, он идет в веб, веб поиск работает хорошо.

Проблемы:
1. Основная проблема - это метка 0.1. Ассесорская разметка не всегда правильна, например в примере с запросом кофе с собой - бургер кинг, модель явно говорит что там 0.1, ведь это не кофейня, но оригинальная метка 1.0
2. Устаревание данных - некоторые данные в датасете устарели.
3. Ошибки в датасете - в датасете много моментов которые необходимо исправить.

Что надо будет сделать
1. Сохранять для классификатора промежуточные выводы роутера и запросы от ллм.
2. Запуск на всем датасете, возможно даже придется убрать субъективный 0.1

### Запуск агента на всех данных.
in development